**Launch Sites Locations Analysis with Folium**

In [2]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

**Task 1: Mark all launch sites on a map**

*First, let's try to add each site's location on a map using site's latitude and longitude coordinates*

*The following dataset with the name spacex_launch_geo.csv is an augmented dataset with latitude and longitude added for each site.*

In [10]:
import pandas as pd
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

*Now, you can take a look at what are the coordinates for each site.*

In [12]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


*Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.*

*We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.*

In [18]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
import folium
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

*We could use folium.Circle to add a highlighted circle area with a text label on a specific coordinate. For example,*

In [21]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

*and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.*

*TODO: Create and add folium.Circle and folium.Marker for each launch site on the site map*

*An example of folium.Circle:*

*folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))*

*An example of folium.Marker:*

*folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))*

In [32]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


*Now, you can explore the map by zoom-in/out the marked areas , and try to answer the following questions:*

*1 Are all launch sites in proximity to the Equator line?*
*2 Are all launch sites in very close proximity to the coast?*
*Also please try to explain your findings.*

 **Task 2: Mark the success/failed launches for each site on the map**

*Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates. Recall that data frame spacex_df has detailed launch records, and the class column indicates if this launch was successful or not*

In [22]:
import pandas as pd
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


*Next, let's create markers for all launch records. If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)*

*Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.*

*Let's first create a MarkerCluster object*

In [29]:
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster()

*TODO: Create a new column in launch_sites dataframe called marker_color to store the marker colors based on the class value*

In [34]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

*TODO: For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster*

In [50]:
# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate over each row in the spacex_df dataframe
for index, record in spacex_df.iterrows():
    # Create a Marker object with its coordinate
    marker = folium.Marker(
        nasa_coordinate,
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    
    # Add the Marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the site map
site_map.add_child(marker_cluster)

**TASK 3: Calculate the distances between a launch site to its proximities**

*Next, we need to explore and analyze the proximities of launch sites.*

*Let's first add a MousePosition on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)*



In [58]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
from folium.plugins import MousePosition
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

*Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.*

In [61]:
from math import sin, cos, sqrt, atan2, radians
def calculate_distance(lat1, lon1, lat2, lon2):
    
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO: Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.*

In [116]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
from geopandas import GeoDataFrame
import shapely.geometry as geom

import geopandas as gpd
import pandas as pd

# Load data into a Pandas DataFrame
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

# Create a GeoDataFrame from the DataFrame
coastline_gdf = gpd.GeoDataFrame(launch_sites_df, geometry=gpd.points_from_xy(launch_sites_df.Long, launch_sites_df.Lat))

# Define launch site latitude and longitude
launch_site_lat = 28.56367
launch_site_lon = -80.57163

# Create a Point object from mouse click coordinates
nasa_coordinate = [29.559684888503615, -95.0830971930759]
mouse_point = geom.Point(nasa_coordinate)

# Find the closest point in the coastline GeoDataFrame
nearest_points = coastline_gdf.sindex.nearest_points(mouse_point)
idx = nearest_points[0][0]
coastline_lat, coastline_lon = coastline_gdf.geometry.iloc[idx].x, coastline_gdf.geometry.iloc[idx].y

# Calculate the distance from the launch site to the closest coastline point
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

AttributeError: 'PyGEOSSTRTreeIndex' object has no attribute 'nearest_points'

In [118]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)

NameError: name 'coastline_lat' is not defined

**TODO: Draw a PolyLine between a launch site to the selected coastline point**

In [121]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
coordinates = [[coastline_lat, coastline_lon], [launch_site_lat, launch_site_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

NameError: name 'coastline_lat' is not defined

**TODO: Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use MousePosition to find the their coordinates on the map first**

In [124]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Define the coordinates of the closest city, railway, highway, etc.
city_coords = [city_lat, city_lon]

# Calculate the distance between the launch site and the city
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create a marker for the city with the distance
city_marker = folium.Marker(
    location=city_coords,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b> (%.2f KM)</div>' % ("Closest City", distance_city),
    )
)

# Add the city marker to the map
site_map.add_child(city_marker)

# Draw a line between the launch site and the city
line = folium.PolyLine(locations=[launch_site_coords, city_coords], weight=1)
site_map.add_child(line)

NameError: name 'city_lat' is not defined